## This a main file. The responses of a chatbot are delivered from here

## Preprocessing the text

In [16]:
%run preprocess_text.ipynb

### Getting operational functions (to book a flight, or cancel a fligth etc)

In [17]:
%run operational_functions.ipynb

In [18]:
import numpy as np


In [19]:
import random
from tensorflow.keras.models import load_model
 

In [20]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)
# create a data structure to hold user context
context = {}

In [21]:

def classify(sentence, words, documents, classes):
    
    ERROR_THRESHOLD = 0.65
    # creatig the bag of words
    p = bow(sentence, words)
    
    d = len(p)
    
    f = len(documents)-2
    
    a = np.zeros([f, d])
    
    #concatenating two lists
    tot = np.vstack((p,a))
    
    #loading the saved model
    model = load_model('model_ChatBot.h5')
    
    
    results = model.predict(tot)[0]
    #print("reults", results)
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability. We can have mode than one response
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    
    # return tuple of intent and probability
    return return_list

documents, classes, words = get_elements(intents)

In [22]:
def response(sentence, userID, context,words, documents, classes, show_details=False):
    results = classify(sentence, words, documents, classes)
    print('Result:',results)
  
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                #print(i['tag'] == results[0][0])
                
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if ((not 'context_filter' in i) or (userID in context and 'context_filter' in i and i['context_filter'] == context[userID])):
                        if results[0][0] == 'book':
                            #res = i['responses']
                            return [(random.choice(i['responses']))]
                        #if show_details: print ('tag:', i['tag'])
                        else:
                            if show_details: print ('tag:', i['tag'])
                            # a random response from the intent
                            return [(random.choice(i['responses']))]
            results.pop(0)


In [23]:

#Creating a funtion for the custom responses. Like different functions will be called for booking, cancellation etx
def print_response(res):
    
    
    if res[0] == "sure, please enter the source and destination":
        print(res[0])
        df = enter_cities()
        print(df)
        get_time()
        print("please enter the class you would like to fly")
        
    elif res[0] == "sure, please enter the PNR number" or res[0] == "no problem, please let me know your reference code" or res[0] == "We do refund the tickets happily. Please enter PNR number":
        print(res[0])
        res = cancel_ticket()
        print(res)
        
    elif res[0]=="sure, enjoy your flight in the lap of luxury" or res[0]=="great, our business class offers range of facilities to make your flight memorable":
        print(res[0])
        ans = book_business_class()
        print(ans)
        print("reserved a business class")
        
    elif res[0]=="sure, our premium economy classes are a great value for your money" or res[0]=="great, our premium economy seats have the biggest leg rooms":
        print(res[0])
        ans = book_premium_economy_class()
        print(ans)
        print("reserved a premium economy for you")
    
    elif res[0]=="sure, we provide the seats at most affordable prices" or res[0]=="great, our seats have biggest leg rooms":
        print(res[0])
        ans = book_economy_class()
        print(ans)
        print("reserved an economy for you")
        
    else:
        for r in res:
            print(r)


In [24]:
res = response("Hello", "123", context,words, documents, classes)
print_response(res)

found in bag: hello
Result: [('greeting', 0.99974483)]
Good to see you again, Welcome to Fly Sutlej


In [25]:
res = response("I want to book a flight","123", context,words, documents, classes)
print_response(res)

found in bag: i
found in bag: want
found in bag: to
found in bag: book
found in bag: a
found in bag: flight
Result: [('book', 0.95589113)]
sure, please enter the source and destination
FromMohali
ToToronto
Enter a date in MM-DD-YYYY format03-12-2021
[  Flight number  Source Destiation   Time
0         PB777  Mohali    Toronto   6:30
1         PB976  Mohali    Toronto  18:30
2         PB315  Mohali    Toronto  23:45]
please enter the flight number you would like to bookPb777
please enter the class you would like to fly


In [26]:
res = response("business class", "123", context,words, documents, classes)
print_response(res)

found in bag: business
found in bag: class
Result: [('business', 0.9999949)]
great, our business class offers range of facilities to make your flight memorable
Booking business class for you
reserved a business class


In [27]:
res = response("How can I pay", "123", context,words, documents, classes)
print_response(res)


found in bag: how
found in bag: can
found in bag: i
found in bag: pay
Result: [('payments', 0.99990606)]
We accept VISA, Mastercard and AMEX


In [28]:
res = response("Thanks!","123", context,words, documents, classes)
print_response(res)

found in bag: thank
Result: [('thanks', 0.99965537)]
have a happy journey


In [29]:
res = response("I need to get my refund back","123", context,words, documents, classes)
print_response(res)

found in bag: i
found in bag: need
found in bag: to
found in bag: get
found in bag: -PRON-
found in bag: refund
found in bag: back
Result: [('cancel', 0.9999926)]
We do refund the tickets happily. Please enter PNR number
Pbasss
The booking for Pbasss has been cancelled
